In [1]:
! pip install pandas
! pip install numpy

zsh:1: command not found: pip


zsh:1: command not found: pip


In [2]:
# Only run this cell if you want to generate a new validation set
import pandas as pd
import numpy as np
df = pd.read_csv("trec/original/train.csv")
# TODO: check w group if we need this
# Remove duplicates from train
df.drop_duplicates(subset='text', keep='first', inplace=True)
# Randomly choose 500 rows to drop 
num_rows_to_drop = 500
np.random.seed(42)
rows_to_drop = np.random.choice(df.index, num_rows_to_drop, replace=False)
# print(rows_to_drop)
# Create a development dataframe from these 500 dropped rows
validation_df = df.loc[rows_to_drop].copy()

# Reset index of development dataframe and export to csv
validation_df.reset_index(drop=True, inplace=True)
validation_df.to_csv("trec/generated/validation.csv")

df_copy = df.copy(deep=True)
# Drop validation rows from original dataset, export as csv
df_copy.drop(rows_to_drop, inplace=True)
df_copy.reset_index(drop=True, inplace=True)
df_copy.to_csv("trec/generated/train.csv")

In [3]:
# Run this cell if you're generating a new validation set for sanity checking
def check_unique_texts(train_csv_file, validation_csv_file):
    train_df = pd.read_csv(train_csv_file)
    validation_df = pd.read_csv(validation_csv_file)

    train_texts = train_df['text']
    validation_texts = validation_df['text']

    common_texts = validation_texts[validation_texts.isin(train_texts)]

    if common_texts.empty:
        print("Validation set and train sets are unique")
    else:
        print("Common values found in the 'text' column:")
        print(common_texts)

train_csv_file = "trec/generated/train.csv"
validation_csv_file = "trec/generated/validation.csv"
check_unique_texts(train_csv_file, validation_csv_file)

Validation set and train sets are unique


In [4]:
# check for duplicates from test - remove if there are any
df = pd.read_csv("trec/original/test.csv")
df.drop_duplicates(subset='text', keep='first', inplace=True)
df.to_csv('trec/generated/test.csv')

In [5]:
train_df=pd.read_csv('trec/original/train.csv')
train_df.drop(columns='label-fine', inplace=True)
test_df=df.copy()
test_df.drop(columns='label-fine', inplace=True)
train_df

,label-coarse,text
0,0,How did serfdom develop in and then leave Russ...
1,1,What films featured the character Popeye Doyle ?
2,0,How can I find a list of celebrities ' real na...
3,1,What fowl grabs the spotlight after the Chines...
4,2,What is the full form of .com ?
...,...,...
5447,1,What 's the shape of a camel 's spine ?
5448,1,What type of currency is used in China ?
5449,4,What is the temperature today ?
5450,4,What is the temperature for cooking ?


In [6]:
import random

classes=train_df['label-coarse'].unique()
random.shuffle(classes)
for i in classes[:2]:
    train_df['label-coarse']=train_df['label-coarse'].apply(lambda x:'OTHERS' if x==i else x)
    test_df['label-coarse']=test_df['label-coarse'].apply(lambda x:'OTHERS' if x==i else x)

train_df['label-coarse'].unique()

array([0, 1, 'OTHERS', 3, 5], dtype=object)

In [7]:
mapping_dict = {item: idx for idx, item in enumerate(set(train_df['label-coarse'].unique()))}
train_df['label-coarse']=train_df['label-coarse'].apply(lambda x:mapping_dict[x])
test_df['label-coarse']=test_df['label-coarse'].apply(lambda x:mapping_dict[x])

test_df

,label-coarse,text
0,4,How far is it from Denver to Aspen ?
1,3,"What county is Modesto , California in ?"
2,2,Who was Galileo ?
3,0,What is an atom ?
4,4,When did Hawaii become a state ?
...,...,...
495,2,Who was the 22nd President of the US ?
496,1,What is the money they use in Zambia ?
497,4,How many feet in a mile ?
498,1,What is the birthstone of October ?


In [8]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from gensim.models import Word2Vec

def iter_df(df):
    for _,x in df.iterrows(): 
        yield tuple(x)

train_iter=iter_df(train_df)

tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [9]:
text_pipeline = lambda x:vocab(tokenizer(x))
label_pipeline = lambda x:int(x)

In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter=iter_df(train_df)
dataloader=DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [11]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [12]:
train_iter=iter_df(train_df)
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [13]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [18]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# Hyperparameters
EPOCHS = 50  # epoch
LR = 0.01  # learning rate
BATCH_SIZE = 32  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = iter_df(train_df), iter_df(test_df)

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset)-500)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  0.33s | valid accuracy    0.936 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  0.35s | valid accuracy    0.936 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  0.35s | valid accuracy    0.934 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  0.37s | valid accuracy    0.934 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  0.37s | valid accuracy    0.934 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  0.36s |

In [19]:
print("Checking the results of test dataset.")
accu_test = evaluate(test_dataloader)
print("test accuracy {:8.3f}".format(accu_test))


Checking the results of test dataset.
test accuracy    0.812
